### Refining the data before curation

#### Costumers

O que é Particionamento no Spark?
É uma técnica de organização de dados que:

Divide fisicamente os arquivos por valores de colunas

Cria subpastas automáticas no formato nome_coluna=valor

Melhora performance em consultas filtradas por essas colunas

In [23]:
# customer_refinement_fixed.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os

# 1. Configuração de caminhos ABSOLUTOS
BASE_DIR = os.path.dirname(os.path.abspath(os.getcwd()))  # Volta um nível para o projeto
INPUT_PATH = os.path.join(BASE_DIR, "data_lake", "parquet", "brazilian_ecommerce_v4", "olist_customers_dataset")
OUTPUT_PATH = os.path.join(BASE_DIR, "data_lake", "refined", "customers")  # Pasta definitiva

# 2. Função de transformação
def transform_customers(df):
    # Limpeza
    df_clean = (df
        .withColumn("customer_city_clean", lower(trim(col("customer_city"))))
        .withColumn("customer_state_clean", upper(trim(col("customer_state"))))
        .dropDuplicates(["customer_id"])
    )
    
    # Cria região (exemplo simplificado)
    return df_clean.withColumn(
        "customer_region",
        when(col("customer_state_clean").isin(["SP", "RJ", "MG", "ES"]), "Sudeste")
        .when(col("customer_state_clean").isin(["RS", "SC", "PR"]), "Sul")
        .when(col("customer_state_clean").isin(["MT", "MS", "GO", "DF"]), "Centro-Oeste")
        .when(col("customer_state_clean").isin(["AM", "PA", "AC", "RO", "RR", "AP", "TO"]), "Norte")
        .when(col("customer_state_clean").isin(["BA", "SE", "AL", "PE", "PB", "RN", "CE", "PI", "MA"]), "Nordeste")
        .otherwise("Outros")  # Apenas para casos extremos não mapeados
    )

# 3. Execução principal
if __name__ == "__main__":
    spark = SparkSession.builder \
        .appName("CustomerRefinement") \
        .config("spark.sql.sources.partitionOverwriteMode", "dynamic") \
        .getOrCreate()

    try:
        # Garante que a pasta de output existe
        os.makedirs(OUTPUT_PATH, exist_ok=True)
        
        print("📂 Lendo dados de:", INPUT_PATH)
        customers_df = spark.read.parquet(INPUT_PATH)
        
        print("🔄 Transformando dados...")
        customers_refined = transform_customers(customers_df)
        
        print("💾 Salvando em:", OUTPUT_PATH)
        (customers_refined.write
            .mode("overwrite")
            .partitionBy("customer_region", "customer_state_clean")  # Partição dupla
            .parquet(OUTPUT_PATH))
        
        # Verificação
        print("✅ Estrutura criada:")
        for root, dirs, files in os.walk(OUTPUT_PATH):
            print(f"📁 {root.replace(BASE_DIR, '...')}")
            for dir in dirs:
                if "=" in dir:  # Mostra apenas pastas de partição
                    print(f"   └── {dir}")

    except Exception as e:
        print(f"❌ Erro: {str(e)}")
    finally:
        spark.stop()

📂 Lendo dados de: c:\Users\pacie\Desktop\Projeto A\data_lake\parquet\brazilian_ecommerce_v4\olist_customers_dataset
🔄 Transformando dados...
💾 Salvando em: c:\Users\pacie\Desktop\Projeto A\data_lake\refined\customers
✅ Estrutura criada:
📁 ...\data_lake\refined\customers
   └── customer_region=Centro-Oeste
   └── customer_region=Nordeste
   └── customer_region=Norte
   └── customer_region=Outras
   └── customer_region=Sudeste
   └── customer_region=Sul
📁 ...\data_lake\refined\customers\customer_region=Centro-Oeste
   └── customer_state_clean=DF
   └── customer_state_clean=GO
   └── customer_state_clean=MS
   └── customer_state_clean=MT
📁 ...\data_lake\refined\customers\customer_region=Centro-Oeste\customer_state_clean=DF
📁 ...\data_lake\refined\customers\customer_region=Centro-Oeste\customer_state_clean=GO
📁 ...\data_lake\refined\customers\customer_region=Centro-Oeste\customer_state_clean=MS
📁 ...\data_lake\refined\customers\customer_region=Centro-Oeste\customer_state_clean=MT
📁 ...\dat

### Geolocation

In [21]:
# geolocation_refinement.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os

# Configuração de caminhos
BASE_DIR = os.path.dirname(os.path.abspath(os.getcwd()))  # Volta um nível
INPUT_PATH = os.path.join(BASE_DIR, "data_lake", "parquet", "brazilian_ecommerce_v4", "olist_geolocation_dataset")
OUTPUT_PATH = os.path.join(BASE_DIR, "data_lake", "refined", "geolocation")

def transform_geolocation_data(df):
    """Transforma os dados de geolocalização com:
    - Limpeza de textos
    - Categorização por região
    - Controle de qualidade
    """
    # Limpeza e padronização
    df_clean = (df
        .withColumn("geolocation_city_clean", lower(trim(col("geolocation_city"))))
        .withColumn("geolocation_state_clean", upper(trim(col("geolocation_state"))))
        .na.drop(subset=["geolocation_zip_code_prefix"])  # Remove registros sem CEP
    )
    
    # Cria região geográfica (mesma lógica dos clientes)
    df_with_region = df_clean.withColumn(
        "region",
        when(col("geolocation_state_clean").isin(["SP", "RJ", "MG", "ES"]), "Sudeste")
        .when(col("geolocation_state_clean").isin(["RS", "SC", "PR"]), "Sul")
        .when(col("geolocation_state_clean").isin(["MT", "MS", "GO", "DF"]), "Centro-Oeste")
        .when(col("geolocation_state_clean").isin(["AM", "PA", "AC", "RO", "RR", "AP", "TO"]), "Norte")
        .otherwise("Nordeste")
    )
    
    # Cria ponto geográfico (lat+lng)
    return df_with_region.withColumn(
        "geo_point",
        format_string("POINT(%s %s)", col("geolocation_lng"), col("geolocation_lat"))
    )

if __name__ == "__main__":
    spark = SparkSession.builder.appName("GeolocationRefinement").getOrCreate()
    
    try:
        # Leitura
        print("📍 Lendo dados de geolocalização...")
        geo_df = spark.read.parquet(INPUT_PATH)
        
        # Transformação
        print("🛠️ Processando dados...")
        geo_refined = transform_geolocation_data(geo_df)
        
        # Escrita com particionamento por região/estado
        print("💾 Salvando em:", OUTPUT_PATH)
        (geo_refined.write
            .mode("overwrite")
            .partitionBy("region", "geolocation_state_clean")
            .option("compression", "snappy")
            .parquet(OUTPUT_PATH))
        
        # Verificação
        print("✅ Dados salvos! Estrutura:")
        print(f"Total de registros: {geo_refined.count()}")
        print("\nAmostra dos dados transformados:")
        geo_refined.show(3)
        
        print("\nExemplo de estrutura gerada:")
        for region in ["Sudeste", "Nordeste", "Sul"]:
            path = os.path.join(OUTPUT_PATH, f"region={region}")
            if os.path.exists(path):
                print(f"📁 {path}")
                print(f"   Estados: {os.listdir(path)}")

    except Exception as e:
        print(f"❌ Falha no processamento: {str(e)}")
        if 'geo_df' in locals():
            print("\nEsquema original:", geo_df.printSchema())
    finally:
        spark.stop()

📍 Lendo dados de geolocalização...
🛠️ Processando dados...
💾 Salvando em: c:\Users\pacie\Desktop\Projeto A\data_lake\refined\geolocation
✅ Dados salvos! Estrutura:
Total de registros: 1000163

Amostra dos dados transformados:
+---------------------------+------------------+-------------------+----------------+-----------------+----------------------+-----------------------+--------+--------------------+
|geolocation_zip_code_prefix|   geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|geolocation_city_clean|geolocation_state_clean|  region|           geo_point|
+---------------------------+------------------+-------------------+----------------+-----------------+----------------------+-----------------------+--------+--------------------+
|                      50760|-8.072588074383969| -34.91359211518264|          recife|               PE|                recife|                     PE|Nordeste|POINT(-34.9135921...|
|                      50740|-8.042627210658422|-3